# AI Capabilities Overview

* ### Text Translation via Open-Source Model API

* ### Knowledge Base Search with Vector Search API (Combined with Assistants)

* ### Image Interpretation (Currently Only via Chat using Individual Files)

* ### Speech to Text via API

* ### Text to Speech - Not Supported

* ### Audio File Processing via Homeport (e.g., with Gemini)

In [ ]:
import mimetypes

import httpx
import json
import base64

## Local config

In [ ]:
API_BASE_URL = "http://localhost:8080/api"
API_KEY = "sk-a1279bbf65dd4b8ab3deff2c0c2f05d3"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
WORKSPACE_ID = "6fa104a8-c0d5-4c0b-9251-52bca4fadd9c"
FOLDER_ID = "1"
FILES_PATH = "./sample_files/"
ASSISTANT_ID = "f21c8a21-ec6a-44f0-bb64-9a7266195e38"

## https://hub.homeport.ai config
<span style="color: red;">Fill the placeholders</span> with your actual values. Use the **list_workspaces** and **list_folders** functions below to get the IDs.

In [ ]:
API_BASE_URL = "https://hub.homeport.ai/api"
API_KEY = "<YOUR_HUB_API_KEY>"
WORKSPACE_ID = "<YOUR_WORKSPACE_ID>"
FOLDER_ID = "<YOUR_FOLDER_ID>"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
FILES_PATH = "./sample_files/"
ASSISTANT_ID = "<YOUR_ASSISTANT_ID>"

In [ ]:
### Helper to create chat message payload
def create_chat_message(model_id: str, prompt: str):
    return {
        "model": f"{model_id}",
        "stream": False,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt}"
                    },
                ]
            }
        ]
    }

### List Workspaces
Run this to get the workspace ID of the created workspace for the next calls.

In [ ]:
def list_workspaces():
    url = f"{API_BASE_URL}/workspaces"

    with httpx.Client() as client:
        response = client.get(
            url=url,
            headers=HEADERS,
        )
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))

list_workspaces()

## List Folders
Use this to get the folder ID of the created folder for the next calls.

In [ ]:
def list_folders():
    url = f"{API_BASE_URL}/folders?workspace_id={WORKSPACE_ID}"

    with httpx.Client() as client:
        response = client.get(
            url=url,
            headers=HEADERS,
        )
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))

list_folders()

### List Files
Use this to check the status of the uploaded file.

In [ ]:
def list_files():
    url = f"{API_BASE_URL}/files?folder_id={FOLDER_ID}"

    with httpx.Client() as client:
        response = client.get(
            url=url,
            headers=HEADERS,
        )
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))

list_files()

### List Models

In [ ]:
async def list_models():
    url = f"{API_BASE_URL}/openai/models"

    async with httpx.AsyncClient() as client:
        response = await client.get(
            url=url,
            headers=HEADERS,
        )
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))

await list_models()

### Upload File

In [ ]:
def upload_file(file_name: str) -> int:
    url = f"{API_BASE_URL}/files"
    file_path = f"{FILES_PATH}/{file_name}"
    file_bytes = None
    mime_type, _ = mimetypes.guess_type(file_path)
    with open(file_path, "rb") as file:
        file_bytes = file.read()
    if not file_bytes:
        raise ValueError("File is empty or could not be read.")

    with httpx.Client() as client:
        response = client.post(
            url=url,
            headers=HEADERS,
            files={"file": (file_name, file_bytes, mime_type)},
            data={
                "workspace_id": WORKSPACE_ID,
                "folder_id": FOLDER_ID,
                "store_vectors": "true",
            }
        )
        response.raise_for_status()
        print(response.json())
        return response.json()["id"]

upload_file("DHL.pdf")

### Call completion endpoint (No RAG capability)
Provide a model ID of an LLM you would like to use. Get available models from the list_models() call.

In [ ]:
def call_completion_api(model_id: str, prompt: str):
    url = f"{API_BASE_URL}/openai/chat/completions"
    payload = create_chat_message(model_id, prompt)

    with httpx.Client() as client:
        response = client.post(
            url=url,
            headers=HEADERS,
            json=payload
        )
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))

call_completion_api("<INSERT MODEL ID (get it by using 'list_models()' above)>", "Translate the following German text to English: 'Hallo zusammen, schön dass Ihr da seit. Wir schauen uns ein paar kleine Beispiel an wie man Homeport per API bedienen kann.'")

### Call Assistant Endpoint (With RAG capability)

In [ ]:
def call_assistant_api(prompt: str):
    timeout = httpx.Timeout(300.0, connect=3.0)
    url = f"{API_BASE_URL}/openai/deployments/{ASSISTANT_ID}/chat/completions?api-version=2024-06-01-preview"
    payload = {
            "stream": False,
            "messages": [
                {
                    "role": "user",
                    "content": f"{prompt}"
                }
            ]
        }

    with httpx.Client(timeout=timeout) as client:
        response = client.post(
            url=url,
            headers=HEADERS,
            json=payload
        )
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))

call_assistant_api("What are the DHL freight eurapid maximum shipment weights?")


### Transcribe Audio File
<span style="color: red;">Note: Only for presentation. Won't work on https://hub.homeport.ai</span>

In [ ]:
def transcribe_audio(file_name: str):
    base64_encoded = None
    with open(FILES_PATH + file_name, "rb") as audio_file:
        file_bytes = audio_file.read()
        base64_encoded = base64.b64encode(file_bytes).decode('utf-8')
    with httpx.Client() as client:
        response = client.post(
            url = f"{API_BASE_URL}/audio/transcribe",
            headers=HEADERS,
            json={
                "base64_audio_url": f"data:audio/wav;base64,{base64_encoded}",
            }
        )
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))

transcribe_audio("icc_audio_demo.mp3")